# Import Data

In [55]:
import pandas as pd
df = pd.read_csv('all_leagues_data.csv', index_col=0)
df

,Home_Team,Away_Team,Result,Link,Season,Round,League
0,Millwall,Southend United,3-1,https://www.besoccer.com/match/millwall-fc/sou...,1995,1,championship
1,Portsmouth,Notts County,2-1,https://www.besoccer.com/match/portsmouth/nott...,1995,1,championship
2,Stoke City,Tranmere Rovers,1-0,https://www.besoccer.com/match/stoke-city/tran...,1995,1,championship
3,Barnsley,Derby County,2-1,https://www.besoccer.com/match/barnsley-fc/der...,1995,1,championship
4,Middlesbrough,Burnley,2-0,https://www.besoccer.com/match/middlesbrough-f...,1995,1,championship
...,...,...,...,...,...,...,...
146493,Frosinone,Pro Vercelli,2-1,https://www.besoccer.com/match/frosinone-calci...,2017,42,serie_b
146494,Perugia,Salernitana,3-2,https://www.besoccer.com/match/perugia/salerni...,2017,42,serie_b
146495,Vicenza,Spezia,0-1,https://www.besoccer.com/match/vicenza-calcio/...,2017,42,serie_b
146496,Ascoli,Ternana Calcio,1-2,https://www.besoccer.com/match/ascoli/ternana-...,2017,42,serie_b


# Data Cleaning and Exploratory Data Analysis

### General Info

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146498 entries, 0 to 146497
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Home_Team  146498 non-null  object
 1   Away_Team  146498 non-null  object
 2   Result     146498 non-null  object
 3   Link       146498 non-null  object
 4   Season     146498 non-null  int64 
 5   Round      146498 non-null  int64 
 6   League     146498 non-null  object
dtypes: int64(2), object(5)
memory usage: 8.9+ MB


Findings:
- No null values
- No glaring issues so far

Need to go deeper into each feature...

### Home_Team

In [63]:
ht = df['Home_Team'].value_counts()
ht

Real Sporting       642
Real Zaragoza       628
Real Valladolid     626
Sochaux             621
Celta               620
                   ... 
SV Babelsberg 03     17
Magdeburg            17
Pau FC               16
UD Logroñés          16
Hallescher FC        11
Name: Home_Team, Length: 538, dtype: int64

### Away_Team

### Result

### Link

### Season

### Round

### League

### Home_Team

### Away_Team

In [ ]:
df.drop(columns=['Link'], inplace=True)
df.drop(df[df['Result'].str.len() != 3].index, inplace=True)
df

In [ ]:
df['Home_Team_Score'] = df['Result'].apply(lambda x: int(x.split('-')[0]))
df['Away_Team_Score'] = df['Result'].apply(lambda x: int(x.split('-')[1]))
df.drop(columns=['Result'], inplace=True)
df

In [ ]:
df['Region'] = df['League']
values_to_update ={
    'Region': {
        'segunda_division': 'Spain',
        'primera_division': 'Spain',
        'serie_b': 'Italy',
        'serie_a': 'Italy',
        'premier_league': 'England',
        'championship': 'England',
        'ligue_1': 'France',
        'ligue_2': 'France',
        '2_liga': 'Germany',
        'bundesliga': 'Germany',
        'eredivisie': 'Netherlands',
        'eerste_divisie': 'Netherlands',
        'primeira_liga': 'Portugal',
        'segunda_liga': 'Portugal'        
        }
}

df = df.replace(values_to_update)
df

In [ ]:
df.corr()

In [ ]:
df_home = df.loc[:,['Home_Team', 'Home_Team_Score', 'Away_Team_Score', 'League', 'Region', 'Season', 'Round']]
df_home.rename(columns={'Home_Team': 'Team', 'Home_Team_Score': 'GS', 'Away_Team_Score': 'GC'}, inplace=True)
df_away = df.loc[:,['Away_Team', 'Away_Team_Score', 'Home_Team_Score', 'League', 'Region', 'Season', 'Round']]
df_away.rename(columns={'Away_Team': 'Team', 'Away_Team_Score': 'GS', 'Home_Team_Score': 'GC'}, inplace=True)
df_teams = pd.concat([df_home, df_away]).reset_index(drop=True)
df_teams

In [ ]:
def win_func(x):
    if x['GS'] > x['GC']:
        return 3
    elif x['GS'] == x['GC']:
        return 1
    else:
        return 0

df_teams['Result'] = df_teams.apply(lambda row: win_func(row), axis=1)
df_teams

In [ ]:
import pickle

In [ ]:
pickle.load(open('elo_dict.pkl', 'rb'))